# Automação de processos com python

## Problemas a serem resolvidos 

- Buscar os dados de investimentos 
- Fazer uma Ánalise dos dados
- Enviar por e-mail

## Passo 1 - Buscar automaticamente os dados da bolsa de valores 

In [21]:
import yfinance

In [22]:
codigo = input("Digite o código da ação: ")
periodo = input("Digite o perído desejado: ")
email = input("Digite para qual e-mail enviar: ")
dados = yfinance.Ticker(codigo).history(periodo)
#print(dados)
fechamento = dados.Close

# Passo 2 - Análise dos dados

## Obter os dados de:
- Cotação máxima
- Cotação mínima
- Cotação atual


In [23]:
maximo = fechamento.max()
minimo = fechamento.min()
atual = fechamento[-1]

print([maximo, minimo, atual])

[42.900001525878906, 35.650001525878906, 38.630001068115234]


C:\Users\Guilherme\AppData\Local\Temp\ipykernel_11108\1324063784.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  atual = fechamento[-1]


# Passo 3 - Fazendo as previsões

- A- Coletar os dados novamente
- B- Tratamento dos dados
- C- Treinar e criar a IA
- D- Pegar os dados e adicionar no pdf


## A - Coletar os dados

In [24]:
# Criando nova variável para os dados dos próximos 7 dias 
tabela=yfinance.Ticker(codigo).history("2y")

In [25]:
tabela.shape


(498, 7)

## B- Tratamento dos dados

In [26]:
# Adicionando index
treinamento = tabela.reset_index()

In [27]:
treinamento[["Date", "Close"]]

,Date,Close
0,2022-04-08 00:00:00-03:00,16.538246
1,2022-04-11 00:00:00-03:00,16.412552
2,2022-04-12 00:00:00-03:00,16.364208
3,2022-04-13 00:00:00-03:00,16.712280
4,2022-04-14 00:00:00-03:00,16.537886
...,...,...
493,2024-04-02 00:00:00-03:00,38.619999
494,2024-04-03 00:00:00-03:00,38.419998
495,2024-04-04 00:00:00-03:00,37.980000
496,2024-04-05 00:00:00-03:00,38.099998


In [28]:
#Selecionando apenas o ano
treinamento["Date"] = treinamento["Date"].dt.date

In [29]:
colunas = ["Date", "Close"]

treinamento[colunas]

treinamento = treinamento[colunas]

In [30]:
treinamento.columns = ["ds", "y"]
treinamento

,ds,y
0,2022-04-08,16.538246
1,2022-04-11,16.412552
2,2022-04-12,16.364208
3,2022-04-13,16.712280
4,2022-04-14,16.537886
...,...,...
493,2024-04-02,38.619999
494,2024-04-03,38.419998
495,2024-04-04,37.980000
496,2024-04-05,38.099998


## C- Criando e treinando a IA

In [31]:
from prophet import Prophet

In [32]:
#Criar o modelo

modelo = Prophet()

# Treinando o modelo 

modelo.fit(treinamento)

22:04:12 - cmdstanpy - INFO - Chain [1] start processing


22:04:12 - cmdstanpy - INFO - Chain [1] done processing


## D- Gerando as previsões

In [33]:
# Fazendo as previsões
tempo = modelo.make_future_dataframe(7)
tempo

,ds
0,2022-04-08
1,2022-04-11
2,2022-04-12
3,2022-04-13
4,2022-04-14
...,...
500,2024-04-11
501,2024-04-12
502,2024-04-13
503,2024-04-14


In [34]:
previsoes = modelo.predict(tempo)
previsoes

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-04-08,19.322504,15.549331,17.933493,19.322504,19.322504,-2.556004,-2.556004,-2.556004,0.570939,0.570939,0.570939,-3.126943,-3.126943,-3.126943,0.0,0.0,0.0,16.766500
1,2022-04-11,19.276187,15.632294,17.987945,19.276187,19.276187,-2.419583,-2.419583,-2.419583,0.546465,0.546465,0.546465,-2.966048,-2.966048,-2.966048,0.0,0.0,0.0,16.856604
2,2022-04-12,19.260748,15.777947,18.024728,19.260748,19.260748,-2.387657,-2.387657,-2.387657,0.547962,0.547962,0.547962,-2.935619,-2.935619,-2.935619,0.0,0.0,0.0,16.873090
3,2022-04-13,19.245309,15.702381,18.098912,19.245309,19.245309,-2.342367,-2.342367,-2.342367,0.574921,0.574921,0.574921,-2.917288,-2.917288,-2.917288,0.0,0.0,0.0,16.902941
4,2022-04-14,19.229869,15.798476,18.251115,19.229869,19.229869,-2.300110,-2.300110,-2.300110,0.610734,0.610734,0.610734,-2.910844,-2.910844,-2.910844,0.0,0.0,0.0,16.929759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2024-04-11,41.538133,38.012661,40.393456,41.538133,41.538133,-2.338587,-2.338587,-2.338587,0.610734,0.610734,0.610734,-2.949321,-2.949321,-2.949321,0.0,0.0,0.0,39.199546
501,2024-04-12,41.596061,38.078781,40.488397,41.596061,41.596061,-2.354008,-2.354008,-2.354008,0.570939,0.570939,0.570939,-2.924947,-2.924947,-2.924947,0.0,0.0,0.0,39.242052
502,2024-04-13,41.653988,36.190277,38.469413,41.653988,41.653988,-4.338118,-4.338118,-4.338118,-1.425511,-1.425511,-1.425511,-2.912607,-2.912607,-2.912607,0.0,0.0,0.0,37.315870
503,2024-04-14,41.711915,36.201632,38.516916,41.711915,41.711915,-4.337433,-4.337433,-4.337433,-1.425509,-1.425509,-1.425509,-2.911924,-2.911924,-2.911924,0.0,0.0,0.0,37.374482


In [35]:
# Pegando as previsoes e os dias 
previsoes = previsoes[["yhat"]].tail(7)
previsoes

,yhat
498,38.936240
499,39.069355
500,39.199546
501,39.242052
502,37.315870
503,37.374482
504,39.394106


In [39]:
# Pegando os últimos 7 valores das previsões
ultimas_previsoes = previsoes[['yhat']].tail(7)

# Atribuindo os valores a variáveis individuais
primeiro = ultimas_previsoes.iloc[-7]['yhat']
segundo = ultimas_previsoes.iloc[-6]['yhat']
terceiro = ultimas_previsoes.iloc[-5]['yhat']
quarto = ultimas_previsoes.iloc[-4]['yhat']
quinto = ultimas_previsoes.iloc[-3]['yhat']
sexto = ultimas_previsoes.iloc[-2]['yhat']
setimo = ultimas_previsoes.iloc[-1]['yhat']

# Passo 4 - Gerando o pdf

In [37]:
from fpdf import FPDF

In [42]:
#criando a variavel
pdf = FPDF()

# Criando a pagina no pdf
pdf.add_page()
pdf.set_font("Arial")
pdf.set_font_size(18)
pdf.image("gkanalises.png", x=0, y=0)


# Adicionando o conteudo
    # Detalhamento 
pdf.text(138, 168, str(round(atual, 2)))
pdf.text(138, 190, str(round(minimo, 2)))
pdf.text(138, 210, str(round(maximo, 2)))

    #Previsoes
pdf.text(20, 255, str(round(primeiro, 2)))
pdf.text(45, 255, str(round(segundo, 2)))
pdf.text(72, 255, str(round(terceiro, 2)))
pdf.text(100, 255, str(round(quarto, 2)))
pdf.text(125, 255, str(round(quinto, 2)))
pdf.text(150, 255, str(round(sexto, 2)))
pdf.text(178, 255, str(round(setimo, 2)))

    #Código e Periodo

pdf.set_font_size(30)
pdf.set_font("Arial", style="B")
pdf.text(33, 110, str(codigo))
pdf.text(140, 110, str(periodo))

# Salvando o pdf
pdf.output(f"GK-{codigo}.pdf")

''

In [43]:
type(atual)

numpy.float64

# Passo 5 - Enviar o código automaticamente 

In [ ]:
import smtplib
import ssl
import mimetypes
from email.message import EmailMessage

In [ ]:
# 1 - Dados do email

password = open("senha", "r").read()
#print(password)
from_email = "guilhermegameplay71@gmail.com"
to_email = email
subject = "Análises diárias"
body = f"""
Prezado leitor.

Seguem em anexos as análises diárias no(s) último(s) {periodo} da ação {codigo} e as previsões dos próximos 7 dias.


Me coloco à disposicao para quaisquer esclarecimentos.

"""

# 2 - Montando a estrutura do e-mail

message = EmailMessage()
message["From"] = from_email
message["To"] = to_email
message["Subject"] = subject

message.set_content(body)
safe = ssl.create_default_context()

# 3 - Adicionando anexo
anexo = f"GK-{codigo}.pdf"
mime_type, mime_subtype = mimetypes.guess_type(anexo)[0].split("/")
with open(anexo, "rb") as a:
    message.add_attachment(
        a.read(),
        maintype = mime_type,
        subtype = mime_subtype,
        filename = anexo
    )


# 4 - Envio do e-mail
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=safe) as smtp:
    #logando no email
    smtp.login(from_email, password)

    #enviando o email
    smtp.sendmail(
        from_email,
        to_email,
        message.as_string()
    )

